### Install the required packages

In [ ]:
#%pip install python-dotenv
#%pip install requests
#%pip install google-genai

### Load the configuration settings

In [ ]:
from google import genai
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "/genai/google/gemini/v1beta", api_key, region_override)

client_config = {
    "base_url": f"https://{environment}/genai/google/gemini",
    "headers": {
        "x-kpmg-charge-code": "123456",
        "Ocp-Apim-Subscription-Key": api_key
    },
    "api_key": api_key
}

client = genai.Client(
  api_key=client_config["api_key"],
  http_options={
    "base_url": client_config["base_url"],
    "headers": client_config["headers"]
  }
)

### Generate Content

In [ ]:
response = client.models.generate_content(
  model="gemini-1.5-flash-002",
  contents="The opposite of hot is"
)
helper.log(response.text)

# Validate the response code
assert len(response.text) > 0

### Generate Content (streaming)

In [ ]:
totalMessage = ''

for chunk in client.models.generate_content_stream(
  model="gemini-1.5-flash-002",
  contents="Write me a two sentence short story"
):
    totalMessage += chunk.text
    helper.log(chunk.text, end='')

assert len(totalMessage) > 0

### Embed Content

In [ ]:
result = client.models.embed_content(
    model="text-embedding-004",
    contents="Hello world",
    config={
        "output_dimensionality": 10
    }
)

helper.log(result.embeddings)

# Validate the response code
assert len(result.embeddings) > 0
assert len(result.embeddings[0].values) > 0

### Batch Embed Content

In [ ]:
texts = [
  "What is the meaning of life?",
  "How much wood would a woodchuck chuck?",
  "How does the brain work?",
]

result = client.models.embed_content(
    model="text-embedding-004",
    contents=texts,
    config={
        "output_dimensionality": 10
    }
)

helper.log(result.embeddings)

assert len(result.embeddings) > 0
assert len(result.embeddings[0].values) > 0

### Generate Answer

In [ ]:
# This functionality is not available in the current version of the API with the currently available models.
# This requires a "thought" capable model e.g. "gemini-2.5-flash-preview-05-20".
# https://ai.google.dev/gemini-api/docs/thinking#summaries

### Count Token

In [ ]:
response = client.models.count_tokens(
    model="models/gemini-1.5-pro-002", contents="The quick brown fox jumps over the lazy dog"
)

helper.log(response)

assert response.total_tokens > 0

### Get Model

In [ ]:
# the google SDK for "GET /models/{deployment_id}" only supports gRPC at this time, not REST. As such, we will use the helper to make the REST call instead of the SDK

deployment_id = "gemini-1.5-pro-002"

# Create the response
response = helper.get("/models/"+ deployment_id)

# Validate the response code
assert response.status_code == 200

### Get List Of Models

In [ ]:
# the google SDK for "GET /models" only supports gRPC at this time, not REST. As such, we will use the helper to make the REST call instead of the SDK

# Create the response
import json

desired_models = ["models/gemini-1.5-flash-002", "models/gemini-1.5-pro-002", "models/text-embedding-004"]
response = helper.get("/models")

# Validate the response code
assert response.status_code == 200

# Parse the JSON response
response_json = json.loads(response.text)

# Extract the "name" attribute from each object in the "models" list
model_names = [model["name"] for model in response_json["models"]]

# Print the matching model names
for name in model_names:
    if name in desired_models:
        print(name)

# Print the models from the array that are not found in model_names
print("\nModels not found:")
for model in desired_models:
    if model not in model_names:
        print(model)